# Quantum Hello World

This is just a copy of the [quantum hello world example of IBM](https://eu-de.quantum.cloud.ibm.com/docs/de/tutorials/hello-world), decorated by some more explicit explanations. 


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorOptions
import qiskit_aer as aer

from matplotlib import pyplot as plt

In [ ]:
# Uncomment the following cell and fill in your token and instance to store your token locally
'''
token = ''
instance = ''
service = QiskitRuntimeService.save_account(
    token = token
    # Do not share your key in public code.
    channel='ibm_cloud', 
    # ibm_cloud is the channel with free access
    instance = instance
    set_as_default=True, # Optionally set these as your default credentials.
  )
'''

In [ ]:
# Create a new circuit with two qubits
qc = QuantumCircuit(2)
 
# Add a Hadamard gate to qubit 0
qc.h(0)
 
# Perform a controlled-X gate on qubit 1, controlled by qubit 0
qc.cx(0, 1)
 
# Return a drawing of the circuit using MatPlotLib ("mpl").
# These guides are written by using Jupyter notebooks, which
# display the output of the last line of each cell.
# If you're running this in a script, use `print(qc.draw())` to
# print a text drawing.
qc.draw("mpl")

In [ ]:
# Set up six different observables.
 
observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]

service = QiskitRuntimeService()
 
backend = service.least_busy(simulator=False, operational=True)
 
# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)
 
isa_circuit.draw("mpl", idle_wires=False)

In [ ]:
# Construct the Estimator instance.
 
estimator = Estimator(mode=backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 5000
 
mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]
 
# One pub, with one circuit to run against five different observables.
job = estimator.run([(isa_circuit, mapped_observables)])
 
# Use the job ID to retrieve your job data later
print(f">>> Job ID: {job.job_id()}")



In [ ]:
# This is the result of the entire submission.  You submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()
 
# This is the result from our single pub, which had six observables,
# so contains information on all six.
pub_result = job.result()[0]


In [ ]:
# Plot the result 
values = pub_result.data.evs
 
errors = pub_result.data.stds
 
# plotting graph
plt.plot(observables_labels, values, "-o")
plt.xlabel("Observables")
plt.ylabel("Values")
plt.show()

## Scale to a larger system
Create a function that returns a QuantumCircuit that prepares an nn-qubit GHZ state (essentially an extended Bell state). The circuit consists of one hadamard gate on q0 followed by n-1 CNOT gates connecting the ith and i+1th qubit. The state can also be illustrated as $1/\sqrt{2} ( |0000\dots 0> + |1111\dots 1>)$.


In [ ]:
def get_qc_for_n_qubit_GHZ_state(n: int) -> QuantumCircuit:
    """This function will create a qiskit.QuantumCircuit (qc) for an n-qubit GHZ state.
 
    Args:
        n (int): Number of qubits in the n-qubit GHZ state
 
    Returns:
        QuantumCircuit: Quantum circuit that generate the n-qubit GHZ state, assuming all qubits start in the 0 state
    """
    if isinstance(n, int) and n >= 2:
        qc = QuantumCircuit(n)
        qc.h(0)
        for i in range(n - 1):
            qc.cx(i, i + 1)
    else:
        raise Exception("n is not a valid input")
    return qc

n = 100
qc = get_qc_for_n_qubit_GHZ_state(n)
#Z operators between qubits to examine the behavior as they get farther apart. 
#Increasingly inaccurate (corrupted) expectation values between distant qubits would reveal the level of noise present.
# ZZII...II, ZIZI...II, ... , ZIII...IZ
operator_strings = [
    "Z" + "I" * i + "Z" + "I" * (n - 2 - i) for i in range(n - 1)
]
print(operator_strings)
print(len(operator_strings))
 
operators = [SparsePauliOp(operator) for operator in operator_strings]

### Step 2: Optimize the problem for the given hardware

In [ ]:
service = QiskitRuntimeService()
 
backend = service.least_busy(
    simulator=False, operational=True, min_num_qubits=100
)
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
 
isa_circuit = pm.run(qc)
isa_operators_list = [op.apply_layout(isa_circuit.layout) for op in operators]

### Execute on hardware

In [ ]:
options = EstimatorOptions()
options.resilience_level = 1
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = "XY4"
 
# Create an Estimator object
estimator = Estimator(backend, options=options)

# Submit the circuit to Estimator
job = estimator.run([(isa_circuit, isa_operators_list)])
job_id = job.job_id()
print(job_id)

### Post-processing


In [ ]:
# data
data = list(range(1, len(operators) + 1))  # Distance between the Z operators
result = job.result()[0]
values = result.data.evs  # Expectation value at each Z operator.
values = [
    v / values[0] for v in values
]  # Normalize the expectation values to evaluate how they decay with distance.
 
# plotting graph
plt.plot(data, values, marker="o", label="100-qubit GHZ state")
plt.xlabel("Distance between qubits $i$")
plt.ylabel(r"$\langle Z_i Z_0 \rangle / \langle Z_1 Z_0 \rangle $")
plt.legend()
plt.show()

### More explicit explanation 

In the plot above, the perfect entanglement should lead to a constant 1 everywhere. You can study that by the hand-crafted simplified example below. In the simulator, the expectation value for each of the Z gate combinations is exactly 1. 

In [ ]:
# A tiny GHZ state manually coded
qc = QuantumCircuit(4)
qc.h(0)
qc.cx(0,1)
qc.cx(1,2)
qc.cx(2,3)
display(qc.draw('mpl'))


#  and the respective measurement operators
observables_labels = ["ZZII", "ZIZI", "ZIIZ"]
observable_operators = [Operator.from_label(label) for label in observables_labels]

backend = aer.AerSimulator(method='automatic')
#transpiling is especially important for newer versions of aer (rearranging the circuit to the given platform, here it's only simulation)
qc_transpiled = transpile(qc, backend)
# add a measurement because we need the state vector later on
qc_transpiled.save_statevector()

#Run the circuit
job = backend.run(qc_transpiled)
result = job.result()
statevector = result.get_statevector()

# calculate the expectation values for ZZII, ZIZI and ZIIZ and convert the complex numpy number to real
exp_vals = [statevector.expectation_value(op).real for op in observable_operators]

plt.plot(observables_labels, exp_vals, marker="o", label="4-qubit GHZ state")
plt.xlabel("Distance between qubits $i$")
plt.ylabel("Expected value")
plt.legend()
plt.show()